In [1]:
from openai import OpenAI
import os
from langchain.prompts import PromptTemplate


##### Set OpenAI API Key as environment variable

In [2]:
openai_key = "update"
os.environ["OPENAI_API_KEY"] = openai_key


##### OpenAI model
Please select model as per number of tokens: <a href="https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo">link</a>
As well update parameters as per your requirements
<ul>
    <li><b>model</b>: OpenAI model</li>
    <li><b>temperature</b></li>
    <li><b>history</b>: old conversation</li>
    <li><b>max_tokens</b>: depending on model and requirements</li>
    <li><b>messages</b>: historical messages</li>
    <li>etc</li>
</ul>

In [3]:
class OpenAIChat:
    """
    Conversational Chatbot using OpenAI
    """
    def __init__(self, messages=[], model="gpt-4", max_tokens=4000, temperature=0, history=""):
        """
        Please feel free to further update as per your requirements
        even prompt and, etc as per your requirements
        """
        self.messages = messages
        self.model = model
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.system_role = "system"
        self.user_role = "user"
        self.history = history
        
        # Please feel free to add a method to update prompts as well.
        self.default_prompt = """You are an AI Conversational Question and Answering Chatbot. Please Answer to the Question using below Context and Coversation History.\nHistory: {history}\nContext: {context}\nQuestion: {question}
        """
        self.openai_client = OpenAI()
        self.prompt = PromptTemplate(template=self.default_prompt, input_variables=["history", "context", "question"])

    def get_model_output(self):
        """
        retrieving the model output
        """
        response = []
        try:
            output = self.openai_client.chat.completions.create(
                model=self.model,
                messages=self.messages,
                max_tokens=self.max_tokens,
                temperature=0, )
            response = list(output)
        except Exception as ex:
            print(f"Error while retrieving OpenAI output: {ex}")
        return response

    def clean_output(self, response):
        """
        cleaning the output and returning the 1st choice
        """
        try:
            for r in response:
                if r[0] == "choices":
                    arr = r[1]
                    for a in arr:
                        out = a.message.content
                        out = str(out)
                        if out.startswith('Answer:'):
                            out = out[8:]
                            out = out.strip()
                        return out
        except Exception as ex:
            print(f"Error while cleaning OpenAI output: {ex}")
        return response

    def add_messages(self, role, content):
        """
        adding it to messages
        """
        self.messages.append({"role": role, "content": content})

    def user_query(self, query="", content="", model=None, max_tokens=None, temperature=0, messages=[], history=None):
        """
        user_query = "Who is Surendra Mallampati?"
        """
        cleanedResponse = ""
        try:
            if query == "":
                print("Please enter query")
                return ""
            
            # updating if any of the parameters are modified 
            if model is not None:
                self.model = model
            if max_tokens is not None:
                self.max_tokens = max_tokens
            if temperature is not None:
                self.temperature = temperature
            if messages:
                self.messages = messages
            if history is not None:
                self.history = history
            
            # creating promp
            prompt = self.prompt.format(history=self.history, context=content, question=query)
            prompt = prompt.strip()
            
            # adding it to user message
            self.add_messages(self.user_role, prompt)
            
            # retrieving output from OpenAI
            openai_response = self.get_model_output()
            
            # cleaning output
            cleanedResponse = self.clean_output(openai_response)
            cleanedResponse = cleanedResponse.strip()
            
            # adding response to messages
            self.add_messages(self.system_role, cleanedResponse)
            
            # updating history
            self.history += "User: " + query + "\n" + "Bot: " + cleanedResponse + "\n"
            return cleanedResponse
        except Exception as ex:
            print(f"Error while querying OpenAI: {ex}")
        return cleanedResponse

    def get_all_messages(self):
        """
        to cache all messages for future uses
        """
        return self.messages
    
    def get_history(self):
        """
        cache history or messages as per your requirements
        """
        return self.history


In [4]:
open_connection = OpenAIChat()
output = open_connection.user_query(query="Hi, I am John!")
output


'Hello John! How can I assist you today?'

##### Sample data

In [6]:
context = 'Kishore Poreddy\nWith over 20 years in the technology industry, Kishore excels at resonating with clients end business goals, translating business requirements into technical scope that meet the clients expectations.He earned a BS degree in Biomedical Engineering and a Bachelor of Law - LLB from Osmania University.\n \n\nSurendra Mallampati\n Surendra wears multiple hats at Fission Labs and leveraging from his 20 plus years of industry experience, the Fission Labs engineering team has pioneered digital transformation across various technology verticals, including AI\n ML, Data Engineering, and Cloud solutions.\n\nHe earned his degree from Andhra University.\n \n\nSrini Majji\n Srini Majji has over 25 years of experience working with enterprises and startups.\n\nAs Head of Finance & Operations, Srini brings synergies among cross functional technical as well as non technical functional groups.\n\nHe has a bachelor\'s of engineering from IIT Madras and an MBA from XLRI.\n \n\nNishita Algubelli\n Nishita Algubelli is the Chief People Officer of Fission Labs.\n\nShe is focused on building a high performance culture while making Fission Labs a "Great Place to Work".\n\nNishita has a Masters Degree in Business Administration from Vellore Institute of Technology.\n \n\nBook a Free Consultation to Unleash the Full Potential of Digital Transformation\n Book a Free Consultation\n Innovation\n Accelerator\n Company\n About Us\n How We Work\n Public Relations & CSR\n Careers\n Privacy Policy\n Contact Us\n Services\n Web & Mobile App Development\n Data Engineering Services\n Cloud Consulting Services\n AI & ML Based Solutions\n Quality Assurance Services\n Salesforce Consulting Services\n Resources\n Blogs\n Case Studies\n E-Books & Whitepapers\n Contact Us\n Enquiry Form\n info@fissionlabs.com\n  2023 Fission Labs - All Rights Reserved\n Fission Labs uses cookies to improve functionality, performance and effectiveness of our communications.\n\nBy continuing to use this site, or by clicking I agree you consent to the use of cookies.\n\nDetailed information on the use of cookies is provided on our \n Cookies Policy\n Got it\n Need Help?'
output = open_connection.user_query(query="Who is Surendra Mallampati?", content=context)
output



'Surendra Mallampati is a key figure at Fission Labs, where he wears multiple hats. With over 20 years of industry experience, he has led the Fission Labs engineering team in pioneering digital transformation across various technology verticals, including AI, ML, Data Engineering, and Cloud solutions. He earned his degree from Andhra University.'

##### Checking if works

In [7]:
output = open_connection.user_query(query="Who am I?", content="")
output

'Based on the conversation history, you are John.'

In [8]:
output = open_connection.user_query(query="Who are you?", content="")
output

"As stated earlier, I am an AI Conversational Question and Answering Chatbot. I'm here to provide information and answer your questions to the best of my ability."

In [9]:
history = open_connection.get_history()
print(history)

User: Hi, I am John!
Bot: Hello John! How can I assist you today?
User: Who is Surendra Mallampati?
Bot: Surendra Mallampati is a key figure at Fission Labs, where he wears multiple hats. With over 20 years of industry experience, he has led the Fission Labs engineering team in pioneering digital transformation across various technology verticals, including AI, ML, Data Engineering, and Cloud solutions. He earned his degree from Andhra University.
User: Who am I?
Bot: Based on the conversation history, you are John.
User: Who are you?
Bot: As stated earlier, I am an AI Conversational Question and Answering Chatbot. I'm here to provide information and answer your questions to the best of my ability.
